<a href="https://colab.research.google.com/github/Tiziri-k/SIC-exercices/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quiz #0801

### "Text Classification with Keras"

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import os
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Slash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Answer the following question by providing Python code:

1). Read in the movie review data from Cornell CS department. Carry out the EDA. <br>
- The data can be found [here](https://www.cs.cornell.edu/people/pabo/movie-review-data). <br>
- Download the “polarity dataset” and unzip. <br>
- Under the "txt_sentoken” folder, there are “pos” and “neg" subfolders. <br>

In [ ]:
reviews = load_files('txt_sentoken/')
x, y = reviews.data, reviews.target

In [ ]:
print("Number of features :", len(x),"\nNumber of target :" ,len(y))

Number of features : 2000 
Number of target : 2000


2). Carry out the data preprocessing: <br>
- Cleaning.
- Stopword removal.

In [ ]:
def clean_text(txt):
    stpw = stopwords.words('english')
    stpw.extend(['www','http','utc'])
    stpw = set(stpw)
    
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-z ]", " ", txt)
    txt = re.sub(r"\b\w{1,3}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    
    return txt

In [ ]:
my_doc =[]
for i in range(len(x)):
    my_doc.append(nltk.sent_tokenize(str(x[i])))

In [ ]:
my_doc[0][0]

'b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80\'s , but lately his films have been very sloppy and the one-liners are getting worse .'

In [ ]:

for i in range(len(my_doc)):
    for j in range(len(my_doc[i])):
      my_doc[i][j] = clean_text(my_doc[i][j])


In [ ]:
len(my_doc)

2000

In [ ]:

X=[]
for i in range(len(my_doc)):
    x=  ' '.join(my_doc[i])
    X.append(x)
X[0]

'a'

3). Carry out label encoding by integers (required form by Keras):

In [ ]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence

In [ ]:

encoding =[]
for i in range(len(X)):
      words = set(text_to_word_sequence(X[i]))
      vocab_size = len(words)
      encoding.append(one_hot(X[i], round(vocab_size*1.3)))

4). Prepare the data for AI: <br>
- Apply the padding.
- Split the data into training and testing.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
padded_inputs =pad_sequences(encoding, padding="post")

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(padded_inputs,y,test_size=0.2,random_state=123)

5). Define the AI model (Embedding + LSTM):

In [ ]:
from tensorflow.keras.layers import SpatialDropout1D

model = Sequential()
model.add(Embedding(1000, 64, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(100, activation='softmax'))

6). Define the optimizer and compile the model:

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


7). Train the model and visualize the summary:

In [ ]:
x_train.shape , y_train.shape

((1600, 1308), (1600,))

In [ ]:
from keras.callbacks import EarlyStopping


In [ ]:
epochs = 5
batch_size = 64
# Code Here
history = model.fit(x_train, y_train, epochs=epochs,
           batch_size=batch_size,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


8). Display the test result (accuracy):

In [ ]:
results =  model.evaluate(x_test , y_test)
print('loss {:.3f}\nAccuracy {:.3f}%'.format(results[0],results[1]*100))